In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from BHDVCStf import BHDVCS

In [4]:
bhdvcs = BHDVCS()

In [5]:
data = pd.read_csv('dvcs_psuedo.csv')

In [ ]:
def produceLHS(data, bhdvcs):
    bh = bhdvcs.GetBHUUxsItsOwn(data['phi_x'], data['F1'], data['F2'], np.array(data[['k', 'QQ', 'x_b', 't']]))
    interference = data['F'] - data['F'] - data['dvcs'] - bh
    bhdvcs.SetKinematics(data['QQ'], data['x_b'], data['t'], data['k'])
    interference /= bhdvcs.Gamma
    interference *= data['QQ']
    interference *= np.abs(data['t'])
    

In [9]:
interference = data['F'] - data['F'] - data['dvcs'] - bhdvcs.GetBHUUxsItsOwn(data['phi_x'], data['F1'], data['F2'], np.array(data[['k', 'QQ', 'x_b', 't']]))



In [10]:
interference

0     -0.186005
1     -0.179169
2     -0.161278
3     -0.138071
4     -0.114739
         ...   
535   -0.030287
536   -0.034515
537   -0.039161
538   -0.043619
539   -0.046949
Length: 540, dtype: float64

In [10]:
bhdvcs.GetBHUUxs(data['phi_x'], data['F1'], data['F2'])

AttributeError: 'BHDVCS' object has no attribute 'qp'